In [1]:
import os
import sys
import re
import random
import chardet

In [2]:
stop_words = [
    "a","about","above","after","again","against","all","am","an","and","any",
    "are","aren't","as","at","be","because","been","before","being","below",
    "between","both","but","by","can't","cannot","could","couldn't","did",
    "didn't","do","does","doesn't","doing","don't","down","during","each",
    "few","for","from","further","had","hadn't","has","hasn't","have","haven't",
    "having","he","he'd","he'll","he's","her","here","here's","hers","herself",
    "him","himself","his","how","how's","i","i'd","i'll","i'm","i've","if","in",
    "into","is","isn't","it","it's","its","itself","let's","me","more","most",
    "mustn't","my","myself","no","nor","not","of","off","on","once","only","or",
    "other","ought","our","ours","ourselves","out","over","own","same","shan't",
    "she","she'd","she'll","she's","should","shouldn't","so","some","such",
    "than","that","that's","the","their","theirs","them","themselves","then",
    "there","there's","these","they","they'd","they'll","they're","they've",
    "this","those","through","to","too","under","until","up","very","was",
    "wasn't","we","we'd","we'll","we're","we've","were","weren't","what",
    "what's","when","when's","where","where's","which","while","who","who's",
    "whom","why","why's","with","won't","would","wouldn't","you","you'd",
    "you'll","you're","you've","your","yours","yourself","yourselves","Subject"
]

In [3]:
def load_labels():
    _labels = {}
    with open('./data/email/train_label.txt') as _fp:
        for _line in _fp.readlines():
            arr = _line.strip().split(' ')
            _labels[arr[1]] = arr[0]
#     test_files = random.sample(list(labels.keys()), 500)
    return _labels


def email_to_list(text, _stop_words):
    _words = re.findall('[a-zA-Z\d]{2,}', text)
    _word_set = []
    for w in _words:
        if w in _stop_words or re.search('\d', w):
            continue
        _word_set.append(w.lower())
    _word_set = list(set(_word_set))
    
    return _word_set
    
    
def load_data_set():
    _emails = {}
    for _fn in  os.listdir('./data/email/train/'):
        if _fn == '.DS_Store':
            continue
        
        ec = chardet.detect(open('./data/email/train/%s' % _fn, "rb").read())['encoding']
        _fp = open('./data/email/train/%s' % _fn, 'r', encoding=ec)
        
        _emails[_fn] = email_to_list(_fp.read(), stop_words)
                    
    return _emails

labels = load_labels()
emails = load_data_set()

In [53]:
test_files = random.sample(list(labels.keys()), 300)

spam_count = 0.0; ham_count = 0.0
word_spam_count = {}; word_ham_count = {}
test_spam_count = 0; test_ham_count = 0

for fn in emails:
    if fn in test_files:
        if int(labels[fn]) == 0:
            test_spam_count += 1
        elif int(labels[fn]) == 1:
            test_ham_count += 1
        continue
        
    if int(labels[fn]) == 0:
        spam_count += 1
        for w in emails[fn]:
            word_spam_count[w] = word_spam_count.get(w, 0) + 1
    elif int(labels[fn]) == 1:
        ham_count += 1
        for w in emails[fn]:
            word_ham_count[w] = word_ham_count.get(w, 0) + 1
            
print('trian num: %s; spam num: %s; ham num: %s' % (len(labels) - 300, spam_count, ham_count))
print('test num: 300; spam num: %s; ham num: %s' % (test_spam_count, test_ham_count))
          
p_y = float(spam_count) / float(ham_count + spam_count)

err_count = 0
err_count_spam = 0
err_count_ham = 0
for fn in test_files:
    ec = chardet.detect(open('./data/email/train/%s' % fn, "rb").read())['encoding']
    fp = open('./data/email/train/%s' % fn, 'r', encoding=ec)
    cur_word_set = email_to_list(fp.read(), stop_words)
    
    p_y0 = p_y; p_y1 = 1.0 - p_y
#     rate = p_y1 / p_y0
    for w in cur_word_set:
        p_xi_y0 = (word_spam_count.get(w, 0.0) + 1.0) / (spam_count + 2.0)
        p_xi_y1 = (word_ham_count.get(w, 0.0) + 1.0) / (ham_count + 2.0)
        
        p_y1 *= p_xi_y1
        p_y0 *= p_xi_y0
        
        if p_y0 < 1e-320 or p_y1 < 1e-320:
            p_y1 *= 1e3
            p_y0 *= 1e3
        
#         rate *= p_xi_y1 / p_xi_y0
#         print(p_y1, p_y0)
    rs = p_y0 / (p_y0 + p_y1)
    if p_y0 == float('inf'):
        rs = 1.0
    
    if rs > 0.9:
        label = 0
    else:
        label = 1
        
    if int(labels[fn]) != label:
        err_count += 1
        if int(labels[fn]) == 0:
            err_count_spam += 1
        elif int(labels[fn]) == 1:
            err_count_ham += 1
#         print(fn, int(labels[fn]), '%.5f' % rs)

print('error spam count: %s; error ham count: %s' % (err_count_spam, err_count_ham))
print('error count: %s; error rate: %.2f' % (err_count, float(err_count)/300))

trian num: 4027; spam num: 1280.0; ham num: 2747.0
test num: 300; spam num: 98; ham num: 202
error spam count: 0; error ham count: 32
error count: 32; error rate: 0.11
